In [7]:
from google.colab import drive
drive.mount('/content/drive')
! pip install open3d

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import torch

import open3d as o3d
import os

import sys
sys.path.append('../')
#sys.path.append('drive/Othercomputers/MacBookPro/')
import NeRF.visualization as v
import utils.data as data


In [2]:
def visualize_dataset(args):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = data.NeRFDataset(args, device=device)
    
    '''mesh = o3d.io.read_triangle_mesh(args.mesh, print_progress=True)
    mesh = o3d.t.geometry.TriangleMesh.from_legacy(mesh)
    scene = o3d.t.geometry.RaycastingScene()
    scene.add_triangles(mesh)
    dataset.create_xnv_dataset(scene)

    for subdataset in dataset.subdatasets:
        imgfiles = [os.path.join(args.dataset_path, subdataset.name, f) for f in sorted(os.listdir(os.path.join(args.dataset_path, subdataset.name))) if f.endswith('JPG') or f.endswith('jpg') or f.endswith('png')]

        for i in range(subdataset.n_images):
            xnv, img = dataset.get_X_target(subdataset.name, i, device=device)
            v.dataset_view_rgb_xnv(img.detach().cpu(), 
                                    (dataset.hwf[0], dataset.hwf[1], 3), 
                                    xnv[..., :3].detach().cpu(), 
                                    xnv[..., 3:6].detach().cpu(), 
                                    xnv[..., 6:].detach().cpu())'''

In [4]:
class Args:
    def __init__(self):
        self.mesh = "../data/lego_llff/meshed-delaunay.ply" #"drive/Othercomputers/MacBookPro/lego_llff/meshed-delaunay.ply"
        self.dataset_path = "../data/lego_llff"#"drive/Othercomputers/MacBookPro/lego_llff"
        self.dataset_type = "llff"
        self.factor = 8
        self.batch_size = 8192
        self.shuffle = True
        self.test = True

visualize_dataset(Args())


['train', 'val']
Loading train - 100 images...
Minifying 8 ../data/lego_llfftrain_8
mogrify -resize 12.5% -format png *.png
Done
	from train_8
	Loaded image data (100, 100, 3, 100) [100.         100.         137.88755316]
	Loaded ../data/lego_llff train
	Changed 1200 items out of 2400 in array of shape (200, 3, 4)
		From (0, 0, 0) to (99, 2, 3)
Loading val - 100 images...
Minifying 8 ../data/lego_llffval_8
mogrify -resize 12.5% -format png *.png
Done
	from val_8


Generating rays:   0%|          | 0/200 [00:00<?, ?pose/s]

	Loaded image data (100, 100, 3, 100) [100.         100.         137.88755316]
	Loaded ../data/lego_llff val
	Changed 1200 items out of 2400 in array of shape (200, 3, 4)
		From (100, 0, 0) to (199, 2, 3)


Generating rays: 100%|██████████| 200/200 [00:02<00:00, 70.49pose/s] 


Creating datasets...
	Computing view dirs for train...
	Creating train dataset with rays (torch.Size([40000, 9])) and images (torch.Size([40000, 3]))
	Computing view dirs for val...
	Creating val dataset with rays (torch.Size([40000, 9])) and images (torch.Size([40000, 3]))
Datasets created successfully

